# Neural Network Gasoline Price Prediction Readme

In [ ]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import requests
import time
import itertools
from statsmodels.tsa.stattools import adfuller
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, LeakyReLU
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from random import shuffle
import keras


In [ ]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import requests
import time
import itertools
from statsmodels.tsa.stattools import adfuller
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, LeakyReLU
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from random import shuffle
import keras


print 'BEGIN @@@@@@@@@@@@@@@@@@'

data = pd.read_csv('Gas_data.csv', index_col = 'Unnamed: 0')
data = data.fillna(method='ffill').dropna()



lookback = 1

testing_data = pd.DataFrame()
for i in range(lookback):
    for j in data.columns:
        if (i == 0) and (j == data.columns[-1]):
            testing_data[str(j)+str(i)] = data[j].shift(0)
        else:
            testing_data[str(j)+str(i)] = data[j].shift(i)
        
testing_data.shape


data['Gas Future'] = data['Gas Future'] / 100
train = data.drop(['DOE Gas', 'Retail Gas'],axis=1)

#train = train.pct_change().fillna(0)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(train[30:].values)


#test_data = series_to_supervised(scaled,10,4)
train_X = testing_data[lookback:3000].values
test_X = testing_data[3000:].values
train_y = data['Retail Gas'][lookback:3000].values
test_y = data['Retail Gas'][3000:].values

# train_y = data['Retail Gas'].pct_change()[0:2500].values
# test_y = data['Retail Gas'][2500:].values
train_X.shape, train_y.shape, test_X.shape, test_y.shape


best_pct = [1.0, 300, 0.25, 256,'mean_squared_error', 20000]
best_correlation = [1.0, 300, 0.25, 256, 'mean_absolute_error', 20000]


combos = [best_pct, best_correlation]


def run_model(lst):
    
	layer_numbers = lst[0]
	layer_size = lst[1]
	dropout_rate = lst[2]
	batch_size = lst[3]
	loss_function = lst[4]
	epochs = lst[5]
	model_number = lst[6]

    
    
	model_name = 'gas_weights_{}'.format(model_number)
    
	print layer_numbers

	if layer_numbers == 1:
		model = Sequential()
		model.add(Dense(layer_size, input_dim = train_X.shape[1]))
		model.add(LeakyReLU(alpha=0.9))
		model.add(Dropout(dropout_rate))
		model.add(Dense(1))
		model.compile(loss=loss_function, optimizer='nadam')

	if layer_numbers == 2:
		print(layer_numbers)
		model = Sequential()
		model.add(Dense(layer_size, input_dim = train_X.shape[1]))
		model.add(LeakyReLU(alpha=0.9))
		model.add(Dropout(dropout_rate))
		model.add(Dense(layer_size/2, activation = 'relu'))
		model.add(Dense(1))
		model.compile(loss=loss_function, optimizer='nadam')
        
        
    
    
	checkpoint = ModelCheckpoint('gas_weights_{}.hdf5'.format(str(model_number)+'_Final'), monitor='val_loss', verbose=1, save_best_only=True, mode='min')
	callbacks_list = [checkpoint]

	history = model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, validation_data=(test_X, test_y), verbose=1,
                    shuffle=False, callbacks=callbacks_list)
    
    
    
	model.load_weights('gas_weights_{}.hdf5'.format(str(model_number)+'_Final'))
	prediction = model.predict(test_X)

	results = pd.DataFrame(zip(prediction.flatten(), test_y), columns = ['pred', 'act'])
	correlation = results.corr().loc['act','pred']
	pct_correct = sum(results.pct_change()['pred'] / results.pct_change()['act'] > 0) * 1. / len(results)
    
	master_dict[model_number] = {}
	master_dict[model_number]['HyperParameters'] = {'Layers':layer_numbers, 'Layer_Size':layer_size, 
                                                    'Dropout':dropout_rate, 
                                                    'Batch_Size': batch_size,
                                                    'Loss_Function': loss_function }
	master_dict[model_number]['Correlation'] = correlation
	master_dict[model_number]['Pct_Correct'] = pct_correct
    
    
    

master_dict = {}
model_number = 2
step = 1
for i in combos:
    #try:
        print 'Running Model {}'.format(model_number)
        run_model(list(i)+[model_number])
        print 'Model {} Successfully Completed'.format(model_number)
        print list(i)+[model_number]
        model_number += step
        
    #except:
        print 'Model Number {} Faileddd'.format(model_number)
        print i
        break
        model_number += step


np.save('master_dict.npy',master_dict)